## Introduction
Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. Click the blue "Edit Notebook" or "Fork Notebook" button at the top of this kernel to begin editing.

## Exploratory Analysis
To begin this exploratory analysis, first use `matplotlib` to import libraries and define functions for plotting the data. Depending on the data, not all plots will be made. (Hey, I'm just a kerneling bot, not a Kaggle Competitions Grandmaster!)

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Imports and Configuration ##

We'll start by importing the packages we used in the exercises and setting some notebook defaults. 

In [2]:
import warnings
from pathlib import Path

import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split

## Load Data ##
The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [3]:
df_train = pd.read_csv('/kaggle/input/diabetic_data.csv', na_values=["?"]) #sep=',\s',
df_train.dataframeName = 'diabetic_data.csv'
nRow, nCol = df_train.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df_train.info()

In [4]:
# Peek at the values
display(df_train)
#display(df_test)
# Display information about dtypes and missing values
display(df_train.info())
display(df_test.info())

In [30]:
col_numeric = [name for name in df_train.select_dtypes("number")]
df_numeric = df_train[col_numeric]

df_categoric = df_train.drop(columns=col_numeric)

In [31]:
df_numeric

In [32]:
df_categoric

## Initial evaluation ##

In [33]:
plt.figure(figsize = (4,6))
ax = df_train.dtypes.value_counts().plot(kind='bar',grid = False,fontsize=20)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+ p.get_width() / 2., height + 1, height, ha = 'center', size = 25)
sns.despine()

**Box Plots**

In [15]:
# Boxplots  of column data
def DrawBoxplotPerColumn(df_num, nGraphShown, nGraphPerRow):
    columnNames = list(df_num)
    nRow, nCol = df_num.shape  # nCol = nFeatures
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow # How many rows of graphs?
    
    plt.figure(num = None, figsize = (1.6 * nGraphPerRow, 2 * nGraphRow), dpi = 100, facecolor = 'w', edgecolor = 'k')
   
    for i in range(min(nCol, nGraphShown)):
        #plt.subplot(9, 3, i + 1)
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        df_num.boxplot([columnNames[i]])

    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
    #plt.savefig('boxplot-HousingDataset.png', dpi=300, bbox_inches='tight')
    
DrawBoxplotPerColumn(df_numeric, 12, 4)

In [17]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    #plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    plt.figure(num = None, figsize = (2 * nGraphPerRow, 3 * nGraphRow), dpi = 140, facecolor = 'w', edgecolor = 'k')
 
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]}')#plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [21]:
#plotPerColumnDistribution(df, nGraphShown, nGraphPerRow)
plotPerColumnDistribution(df_train.iloc[:,15:], 18, 4)



In [41]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


## Missing Value ##  
**visualization**

In [14]:
df_categoric

In [21]:
labels, frequency = np.unique(df_categoric.iloc[:,4:5], return_counts = True)

In [22]:
labels

In [23]:
frequency

In [34]:
msno.bar(df_numeric)

In [35]:
msno.bar(df_categoric)

In [36]:
df_numeric.info()

In [39]:
isnull_series = df_categoric.iloc[:,:9].isnull().sum()
isnull_series[isnull_series >= 20].sort_values(ascending=False)

plt.figure(figsize = (20,10))
#ax = isnull_series[isnull_series >= 20].sort_values(ascending=False).plot(kind='bar', grid = False,fontsize=20,color='grey')
ax = isnull_series[isnull_series >= 1].sort_values(ascending=False).plot(kind='bar', grid = False,fontsize=20)
plt.legend(loc = 'upper right')
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+ p.get_width() / 2., height + 5, height, ha = 'center', size = 20)
sns.despine()

In [ ]:
# df_train[col_numeric] df_numeric , df_categoric 




In [42]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [43]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


Now you're ready to read in the data and use the plotting functions to visualize the data.

### Let's check 1st file: /kaggle/input/diabetic_data.csv

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# diabetic_data.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('/kaggle/input/diabetic_data.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'diabetic_data.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:

In [ ]:
df1.head(5)

Distribution graphs (histogram/bar graph) of sampled columns:

In [44]:
df1=df_train

In [ ]:
plotPerColumnDistribution(df1, 10, 5)

Correlation matrix:

In [ ]:
plotCorrelationMatrix(df1, 8)

Scatter and density plots:

In [ ]:
plotScatterMatrix(df1, 20, 10)

## Conclusion
This concludes your starter analysis! To go forward from here, click the blue "Edit Notebook" button at the top of the kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!